## * 아래 2칸 실행

In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk

import matplotlib as mpl
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()

In [3]:
# 경로 변수 (고정)

dataLoc = '../_data/'
trainfile = 'train_03_997.csv'
old_trainfile = 'train.csv'
testfile = 'test_clean.csv'
old_testfile = 'test.csv'
train_171016 = '171016_dow_hr_pas_L2_dur_only.csv'


## 원본 `train.csv` (전처리 이전)

In [54]:
# train_03_997.csv
taxi_df = pd.read_csv(dataLoc + old_trainfile)
#taxi_df['p_dt'] = pd.to_datetime(taxi_df['p_dt'])
taxi_df.isnull().sum().sum()

0

### 요일, hr, 탑승객 수, 직선거리, duration 변수만 남긴 `train.csv`

In [4]:
# 171016_dow_hr_pas_L2_dur_only.csv
main_df = pd.read_csv(dataLoc + train_171016)
main_df.head()

,dow,hr,pas,L2,dur
0,Sa,23,1,0.069424,1454
1,Sa,23,1,0.057791,1409
2,Sa,23,2,0.066698,1081
3,Sa,23,1,0.018223,800
4,Sa,23,1,0.027198,1151


### OLS: duration = 요일 + 시간 + 탑승객 + 거리 // (상수없음) 

In [9]:
model0 = sm.OLS.from_formula("dur ~ C(dow) + C(hr) + C(pas) + L2 - 1", data=main_df)

In [10]:
print(model0.fit().summary())

                            OLS Regression Results                            
Dep. Variable:                    dur   R-squared:                       0.611
Model:                            OLS   Adj. R-squared:                  0.611
Method:                 Least Squares   F-statistic:                 2.975e+04
Date:                Tue, 17 Oct 2017   Prob (F-statistic):               0.00
Time:                        19:18:38   Log-Likelihood:            -4.9786e+06
No. Observations:              682598   AIC:                         9.957e+06
Df Residuals:                  682561   BIC:                         9.958e+06
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
C(dow)[F]     516.4183    102.779      5.025      

### OLS: duration = 요일 + 시간 + 거리 // (상수있음) 

In [7]:
model1 = sm.OLS.from_formula("dur ~ C(dow) + C(hr) + L2", data=main_df)

In [8]:
print(model1.fit().summary())

                            OLS Regression Results                            
Dep. Variable:                    dur   R-squared:                       0.611
Model:                            OLS   Adj. R-squared:                  0.610
Method:                 Least Squares   F-statistic:                 3.566e+04
Date:                Tue, 17 Oct 2017   Prob (F-statistic):               0.00
Time:                        19:17:43   Log-Likelihood:            -4.9789e+06
No. Observations:              682598   AIC:                         9.958e+06
Df Residuals:                  682567   BIC:                         9.958e+06
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      335.2862      2.529    132.598   

### OLS: duration = 요일 + 시간 + 거리 // (상수없음) 

In [5]:
model2 = sm.OLS.from_formula("dur ~ C(dow) + C(hr) + L2 - 1", data=main_df)

In [7]:
res2 = model2.fit()
print(res2.summary())

                            OLS Regression Results                            
Dep. Variable:                    dur   R-squared:                       0.611
Model:                            OLS   Adj. R-squared:                  0.610
Method:                 Least Squares   F-statistic:                 3.566e+04
Date:                Thu, 19 Oct 2017   Prob (F-statistic):               0.00
Time:                        20:11:55   Log-Likelihood:            -4.9789e+06
No. Observations:              682598   AIC:                         9.958e+06
Df Residuals:                  682567   BIC:                         9.958e+06
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
C(dow)[F]     335.2862      2.529    132.598      

In [8]:
sm.stats.anova_lm(res2, typ=2)

,sum_sq,df,F,PR(>F)
C(dow),3.495925e+09,7.0,3.939208e+03,0.0
C(hr),5.850627e+09,23.0,2.006409e+03,0.0
L2,1.321012e+11,1.0,1.041961e+06,0.0
Residual,8.653673e+10,682567.0,NaN,NaN


--------
## \*\* 분석용 간결한 df 만들기 \*\*
### 1. df 복사

In [4]:
copy_df = taxi_df.copy()  # 분석 필요한 df명 넣기
copy_df.head()

,p_dt,pas,p_lng,p_lat,d_lng,d_lat,dur
0,2016-04-30 23:59:00,1,-73.987793,40.724792,-73.975616,40.656445,1454
1,2016-04-30 23:59:00,1,-73.957596,40.717770,-73.951424,40.775230,1409
2,2016-04-30 23:59:00,2,-74.000954,40.742031,-73.947708,40.782200,1081
3,2016-04-30 23:58:00,1,-73.985733,40.738258,-73.993179,40.754890,800
4,2016-04-30 23:58:00,1,-74.006615,40.740650,-73.985619,40.723362,1151


### 2. `copy_df`에 두번째 열로 `hr`값 삽입

In [5]:
copy_df.insert(loc=1, column='hr', value=copy_df['p_dt'].dt.hour)

### 3. `copy_df`에 두번째 열로 요일값 삽입

In [6]:
dow_conv = {0 : 'M', 1 : 'T', 2 : 'W', 3 : 'R', 4 : 'F', 5 : 'Sa', 6 : 'Su'}

copy_df.insert(loc=1, column='dow', value=copy_df['p_dt'].dt.dayofweek)

In [7]:
copy_df = copy_df.replace({'dow' : dow_conv})

In [8]:
copy_df.head()

,p_dt,dow,hr,pas,p_lng,p_lat,d_lng,d_lat,dur
0,2016-04-30 23:59:00,Sa,23,1,-73.987793,40.724792,-73.975616,40.656445,1454
1,2016-04-30 23:59:00,Sa,23,1,-73.957596,40.717770,-73.951424,40.775230,1409
2,2016-04-30 23:59:00,Sa,23,2,-74.000954,40.742031,-73.947708,40.782200,1081
3,2016-04-30 23:58:00,Sa,23,1,-73.985733,40.738258,-73.993179,40.754890,800
4,2016-04-30 23:58:00,Sa,23,1,-74.006615,40.740650,-73.985619,40.723362,1151


In [36]:
copy_df.insert(loc=1, column='m_d', value=np.vectorize(str)(copy_df['p_dt'].dt.month*100 + copy_df['p_dt'].dt.day))

In [35]:
del copy_df['m_d']

In [38]:
copy_df.tail()

,p_dt,m_d,dow,hr,pas,p_lng,p_lat,d_lng,d_lat,dur
682593,2016-01-01 00:01:00,101,F,0,1,-73.982292,40.751331,-73.991341,40.750340,280
682594,2016-01-01 00:01:00,101,F,0,5,-73.965279,40.801041,-73.947479,40.815170,408
682595,2016-01-01 00:01:00,101,F,0,1,-73.970108,40.759800,-73.989357,40.742989,736
682596,2016-01-01 00:00:00,101,F,0,5,-73.981743,40.719158,-73.938828,40.829182,849
682597,2016-01-01 00:00:00,101,F,0,1,-73.985085,40.747166,-73.958038,40.717491,1294


### 4. `p_dt`열 삭제

In [39]:
# 한번만 실행 가능
del copy_df['p_dt']
copy_df.head()

,m_d,dow,hr,pas,p_lng,p_lat,d_lng,d_lat,dur
0,430,Sa,23,1,-73.987793,40.724792,-73.975616,40.656445,1454
1,430,Sa,23,1,-73.957596,40.717770,-73.951424,40.775230,1409
2,430,Sa,23,2,-74.000954,40.742031,-73.947708,40.782200,1081
3,430,Sa,23,1,-73.985733,40.738258,-73.993179,40.754890,800
4,430,Sa,23,1,-74.006615,40.740650,-73.985619,40.723362,1151


### 5. 거리 (L2 norm) 변수 생성 및 삽입

In [40]:
def dist(plng, plat, dlng, dlat):
    return ((dlng-plng)**2 + (dlat-plat)**2) ** 0.5

In [41]:
copy_df.insert(loc=7, column='L2', value=np.vectorize(dist)(\
                copy_df['p_lng'], copy_df['p_lat'],\
                copy_df['d_lng'], copy_df['d_lat']))

copy_df.head()

,m_d,dow,hr,pas,p_lng,p_lat,d_lng,L2,d_lat,dur
0,430,Sa,23,1,-73.987793,40.724792,-73.975616,0.069424,40.656445,1454
1,430,Sa,23,1,-73.957596,40.717770,-73.951424,0.057791,40.775230,1409
2,430,Sa,23,2,-74.000954,40.742031,-73.947708,0.066698,40.782200,1081
3,430,Sa,23,1,-73.985733,40.738258,-73.993179,0.018223,40.754890,800
4,430,Sa,23,1,-74.006615,40.740650,-73.985619,0.027198,40.723362,1151


### 6. 경도/위도 열 삭제

In [42]:
# 4개 열 전부 삭제
todel = ['p_lng', 'p_lat', 'd_lng', 'd_lat']

for thing in todel:
    del copy_df[thing]

In [43]:
copy_df.head()

,m_d,dow,hr,pas,L2,dur
0,430,Sa,23,1,0.069424,1454
1,430,Sa,23,1,0.057791,1409
2,430,Sa,23,2,0.066698,1081
3,430,Sa,23,1,0.018223,800
4,430,Sa,23,1,0.027198,1151


### 7. 필요한 경로로 `csv`파일 저장 (끝)

In [44]:
copy_df.to_csv("../_data/171018_md_dow_hr_pas_L2_dur_only.csv", index=False)

---
## * 각종 분석 시도

In [3]:
copy_df = pd.read_csv("../_data/171018_md_doy_dow_hr_pas_L2_dur_only.csv")

In [46]:
model3 = sm.OLS.from_formula("dur ~ C(m_d) + C(dow) + C(hr) + L2 - 1", data=copy_df)

In [47]:
print(model3.fit().summary())

                            OLS Regression Results                            
Dep. Variable:                    dur   R-squared:                       0.617
Model:                            OLS   Adj. R-squared:                  0.617
Method:                 Least Squares   F-statistic:                     7622.
Date:                Wed, 18 Oct 2017   Prob (F-statistic):               0.00
Time:                        20:36:18   Log-Likelihood:            -4.9735e+06
No. Observations:              682598   AIC:                         9.947e+06
Df Residuals:                  682453   BIC:                         9.949e+06
Df Model:                         144                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
C(m_d)[101]    203.4644      5.485     37.093   

In [48]:
dow_conv = {0 : 'M', 1 : 'T', 2 : 'W', 3 : 'R', 4 : 'F', 5 : 'Sa', 6 : 'Su'}

In [49]:
dow_chars = list(dow_conv.values())
dow_chars

['M', 'T', 'W', 'R', 'F', 'Sa', 'Su']

In [50]:
dow_dfs = list(map(lambda x: copy_df[copy_df['dow'] == x], dow_chars))

dow_dfs[0].head()

,m_d,dow,hr,pas,L2,dur
29210,425,M,23,1,0.089469,1468
29211,425,M,23,1,0.027976,530
29212,425,M,23,1,0.028892,332
29213,425,M,23,5,0.006448,332
29214,425,M,23,2,0.011086,124


In [52]:
dow_models = list(map(lambda x: sm.OLS.from_formula("dur ~ C(m_d) + C(hr) + L2 - 1", data=x), dow_dfs))
for idx, model in enumerate(dow_models):
    print(dow_conv[idx], 'model')
    print(model.fit().summary())

M model
                            OLS Regression Results                            
Dep. Variable:                    dur   R-squared:                       0.664
Model:                            OLS   Adj. R-squared:                  0.664
Method:                 Least Squares   F-statistic:                     4242.
Date:                Wed, 18 Oct 2017   Prob (F-statistic):               0.00
Time:                        20:36:47   Log-Likelihood:            -6.2086e+05
No. Observations:               85953   AIC:                         1.242e+06
Df Residuals:                   85912   BIC:                         1.242e+06
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
C(m_d)[104]    63.3317      9.071      6.9

                            OLS Regression Results                            
Dep. Variable:                    dur   R-squared:                       0.627
Model:                            OLS   Adj. R-squared:                  0.627
Method:                 Least Squares   F-statistic:                     4237.
Date:                Wed, 18 Oct 2017   Prob (F-statistic):               0.00
Time:                        20:36:48   Log-Likelihood:            -7.3827e+05
No. Observations:              100658   AIC:                         1.477e+06
Df Residuals:                  100617   BIC:                         1.477e+06
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
C(m_d)[107]   105.3991      8.787     11.995      

                            OLS Regression Results                            
Dep. Variable:                    dur   R-squared:                       0.600
Model:                            OLS   Adj. R-squared:                  0.600
Method:                 Least Squares   F-statistic:                     3940.
Date:                Wed, 18 Oct 2017   Prob (F-statistic):               0.00
Time:                        20:36:48   Log-Likelihood:            -7.7726e+05
No. Observations:              107869   AIC:                         1.555e+06
Df Residuals:                  107827   BIC:                         1.555e+06
Df Model:                          41                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
C(m_d)[102]   283.3622      6.520     43.458      

In [20]:
print(model3.fit().summary())

                            OLS Regression Results                            
Dep. Variable:                    dur   R-squared:                       0.617
Model:                            OLS   Adj. R-squared:                  0.617
Method:                 Least Squares   F-statistic:                     7622.
Date:                Wed, 18 Oct 2017   Prob (F-statistic):               0.00
Time:                        20:11:43   Log-Likelihood:            -4.9735e+06
No. Observations:              682598   AIC:                         9.947e+06
Df Residuals:                  682453   BIC:                         9.949e+06
Df Model:                         144                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
C(doY)[1]      203.4644      5.485     37.093   